# Running fill-spill-merge on REMA
This version works. 

fsm_on_rema.ipynb seems to be broken and it is over complicated. 

This notebook defines two function, one for running FSM for a given DEM and melt input file, and anothe for creating the melt input file. 

It then runs FSM multiple times each time using a different melt input file.

In [14]:
import rasterio
from matplotlib import pyplot as plt
from urllib.request import urlretrieve
import numpy as np
import xarray as xr
import os
import rasterio
import rioxarray
import panel.widgets as pnw
import hvplot.xarray # noqa 
from typing import Optional, Tuple

%matplotlib widget

## Define a function for running fill-spill-merge


In [19]:
def fsm(dem_filename: str, 
        prefix: str = "fsm_results/rema_tests/test-2", 
        uniform_melt: Optional[float] = None, 
        melt_filename: Optional[str] = None, 
        sea_level: float = 0.0,
        path_to_fsm: str = "../../../Barnes2020-FillSpillMerge/build/fsm.exe") -> xr.DataArray:
    
    print("running fill-spill-merge...")
    

    if uniform_melt is not None:
        os.system(f"{path_to_fsm} {dem_filename} {prefix} {sea_level} --swl={uniform_melt}")

    if melt_filename is not None:
        os.system(f"{path_to_fsm} {dem_filename} {prefix} {sea_level} --swf={melt_filename}")

    if uniform_melt is None and melt_filename is None:
        raise ValueError("Must specify either uniform_melt or water_input_file") 


    print("loading results...")
    #surface_height = rioxarray.open_rasterio(prefix + "surface-height.tif").squeeze()
    water_depth = rioxarray.open_rasterio(prefix + "-wtd.tif").squeeze()
    #labels = rioxarray.open_rasterio(prefix + "label.tif").squeeze()

    return water_depth

## Define a function for creating a melt map 

In [20]:
def rectangular_melt_region(dem: xr.DataArray, 
                            melt_magnitude: float, 
                            xmin: float = 815000, 
                            xmax: float = 820000, 
                            ymin: float = 1.93e6, 
                            ymax: float = 1.935e6,
                            melt_filename = "rema_subsets/water_input_file_3.tif")\
      -> Tuple[xr.DataArray, str, Tuple[float, float, float, float]]:
    
    # starting with the DEM, make a melt file with a rectangular region of non-zero melt
    melt = dem.copy().squeeze()
    melt[:, :] = 0
    melt.loc[ymax:ymin, xmin:xmax] = melt_magnitude  # ymin and ymax are flipped because the y-axis is flipped
    
    # Save the melt file
    melt.rio.to_raster(melt_filename)

    melt_bounds = (xmin, ymin, xmax, ymax)
    return melt, melt_filename, melt_bounds

## Run FSM with different melt magnitudes

In [73]:
    
def loop_over_melt_magnitudes(dem_filename = "rema_subsets/dem_small_2.tif",
                            xmin: float = 815000, 
                            xmax: float = 820000, 
                            ymin: float = 1.93e6, 
                            ymax: float = 1.935e6,):
    # Load the DEM
    dem = rioxarray.open_rasterio(dem_filename, chunks={})
    dem = dem.squeeze()

    # Create a list of melt_magnitudes to loop over
    melt_magnitudes = np.linspace(start=1, stop=30, num=2)

    print(f"running fsm for melt_magnitudes: {melt_magnitudes} m")
    water_depths = []    # a list for holding multiple water_depth xr.DataArrays
    melts = []           # a list for holding multiple melt xr.DataArrays 
    bounds_list = []     # a list for holding the bounds of the rectangular melt regions 

    # loop over the melt_magnitudes 
    for melt_magnitude in melt_magnitudes:
        melt, melt_filename, bounds = rectangular_melt_region(dem, melt_magnitude, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)   # write a melt tiff file to disk with a rectangular region of non-zero melt. Also return the melt xr.DataArray and the filename of the melt tiff file.
        
        bounds_list.append(bounds)                                                   # append the bounds of the rectangular melt region to the list 'bounds_list'
        melts.append(melt)                                                           # append the melt xr.DataArray to the list 'melts'
        water_depths.append(fsm(dem_filename, melt_filename=melt_filename))          # append the water_depth xr.DataArray to the list 'water_depths'


    # Create a new xarray from the array 'melt_magnitudes' so we can pass both the values of melt_magnitudes and the name 'melt_magnitudes' to xr.concat 
    melt_magnitudes_xr = xr.DataArray(melt_magnitudes, dims='melt_mag', name='melt_mag') 

    # contatenate
    water_depth = xr.concat(water_depths, dim=melt_magnitudes_xr)
    melt = xr.concat(melts, dim=melt_magnitudes_xr)

    # name the xr.DataArrays
    water_depth.name = 'water_depth'
    dem.name = 'dem'
    melt.name = 'melt'

    # merge the xr.DataArrays into a xr.Dataset
    results = xr.merge([water_depth, dem, melt])
    results = results.drop_vars('band')   # drop this unneeded variable

    # add information about the coordinates and variabels in attributes
    results.melt_mag.attrs = {'units': 'meters', 'long_name': 'melt magnitude', 'description': 'the magnitude of the melt in the rectangular region'}
    results.melt.attrs = {'units': 'meters', 'long_name': 'surface melt', 'description': 'the surface melt as a function of x and y'}

    # put bounds list into an xarray
    bounds_list = np.array(bounds_list)
    results['bounds'] = xr.DataArray(bounds_list, dims=['melt_mag', 'bounds_index'], name='bounds')
    results.bounds.attrs = {'long_name': 'bounds of the rectangular melt region', 'description': 'the bounds of the rectangular melt region: (xmin, ymin, xmax, ymax)'}


    # add the center of mass of the water (see centroid_test.ipynb for notes on ths method)
    weights = results.water_depth.fillna(0)
    results['x_center'] = results.x.weighted(weights).mean(dim = ['x', 'y'])
    results['y_center'] = results.y.weighted(weights).mean(dim = ['x', 'y'])
    results.x_center.attrs = {'long_name': 'x coordinate of the center of mass', 'description': 'the x coordinate of the center of mass of the water, i.e. the depth-weighted centroid'}
    results.y_center.attrs = {'long_name': 'y coordinate of the center of mass', 'description': 'the y coordinate of the center of mass of the water, i.e. the depth-weighted centroid'}
    #results['distance_to_center_of_mass'] = np.sqrt((result.x_center-) + ()**2)
    return results



In [63]:
results = loop_over_melt_magnitudes(ymin=1.91e6, ymax=1.915e6)

running fsm for melt_magnitudes: [ 1. 30.] m
running fill-spill-merge...
m Input DEM           = rema_subsets/dem_small_2.tif
m Output prefix       = fsm_results/rema_tests/test-2
m Surface water level = nan
m Surface water file  = rema_subsets/water_input_file_3.tif
m Ocean level         = 0
m Data width  = 1024
m Data height = 3072
m Data cells  = 3145728


Finished.
IO time   = 0.0693452 s
Calc time = 2.32362 s
loading results...
running fill-spill-merge...
m Input DEM           = rema_subsets/dem_small_2.tif
m Output prefix       = fsm_results/rema_tests/test-2
m Surface water level = nan
m Surface water file  = rema_subsets/water_input_file_3.tif
m Ocean level         = 0
m Data width  = 1024
m Data height = 3072
m Data cells  = 3145728


Finished.
IO time   = 0.053254 s
Calc time = 2.32099 s
loading results...


In [70]:
results = loop_over_melt_magnitudes(ymin=1.91e6, ymax=1.915e6)
hvplot.extension('bokeh')
coarse = results.coarsen(x=10, y=10, boundary='trim').mean()
coarse.water_depth.hvplot(x ='y', y = 'x', cmap='Blues', clim=(0,1), width = 1200, height = 300, aspect='equal')\
    * coarse.dem.hvplot.contour(x ='y', y = 'x',levels = 40, cmap='hot')\
    * coarse.melt.hvplot.contour(x ='y', y = 'x', levels=[0.0, 0.0])\
    * coarse.hvplot.scatter(x = 'y_center', y = 'x_center', color = 'green', size = 400, marker = '*')


running fsm for melt_magnitudes: [ 1. 30.] m
running fill-spill-merge...
m Input DEM           = rema_subsets/dem_small_2.tif
m Output prefix       = fsm_results/rema_tests/test-2
m Surface water level = nan
m Surface water file  = rema_subsets/water_input_file_3.tif
m Ocean level         = 0
m Data width  = 1024
m Data height = 3072
m Data cells  = 3145728


Finished.
IO time   = 0.0646395 s
Calc time = 2.24364 s
loading results...
running fill-spill-merge...
m Input DEM           = rema_subsets/dem_small_2.tif
m Output prefix       = fsm_results/rema_tests/test-2
m Surface water level = nan
m Surface water file  = rema_subsets/water_input_file_3.tif
m Ocean level         = 0
m Data width  = 1024
m Data height = 3072
m Data cells  = 3145728


Finished.
IO time   = 0.0481052 s
Calc time = 2.52807 s
loading results...


BokehModel(combine_events=True, render_bundle={'docs_json': {'de39bb77-1b68-4436-bc17-690c56735f13': {'version…

In [74]:
results = loop_over_melt_magnitudes(dem_filename = "/Users/jkingslake/Documents/science/meltwater_routing/big_data/boudouin_west_1.tif", xmin = 9e5, xmax = 9.05e5, ymin = 1.875e6, ymax = 1.88e6)
results = loop_over_melt_magnitudes(ymin=1.91e6, ymax=1.915e6)
hvplot.extension('bokeh')
coarse = results.coarsen(x=10, y=10, boundary='trim').mean()
coarse.water_depth.hvplot(x ='y', y = 'x', cmap='Blues', clim=(0,1), width = 1200, height = 300, aspect='equal')\
    * coarse.dem.hvplot.contour(x ='y', y = 'x',levels = 40, cmap='hot')\
    * coarse.melt.hvplot.contour(x ='y', y = 'x', levels=[0.0, 0.0])\
    * coarse.hvplot.scatter(x = 'y_center', y = 'x_center', color = 'green', size = 400, marker = '*')

running fsm for melt_magnitudes: [ 1. 30.] m
running fill-spill-merge...
m Input DEM           = /Users/jkingslake/Documents/science/meltwater_routing/big_data/boudouin_west_1.tif
m Output prefix       = fsm_results/rema_tests/test-2
m Surface water level = nan
m Surface water file  = rema_subsets/water_input_file_3.tif
m Ocean level         = 0
m Data width  = 2500
m Data height = 2500
m Data cells  = 6250000


libc++abi: terminating due to uncaught exception of type std::runtime_error: No OCEAN cells found, could not make a DepressionHierarchy!
libc++abi: terminating due to uncaught exception of type std::runtime_error: No OCEAN cells found, could not make a DepressionHierarchy!


loading results...
running fill-spill-merge...
m Input DEM           = /Users/jkingslake/Documents/science/meltwater_routing/big_data/boudouin_west_1.tif
m Output prefix       = fsm_results/rema_tests/test-2
m Surface water level = nan
m Surface water file  = rema_subsets/water_input_file_3.tif
m Ocean level         = 0
m Data width  = 2500
m Data height = 2500
m Data cells  = 6250000
loading results...
running fsm for melt_magnitudes: [ 1. 30.] m
running fill-spill-merge...
m Input DEM           = rema_subsets/dem_small_2.tif
m Output prefix       = fsm_results/rema_tests/test-2
m Surface water level = nan
m Surface water file  = rema_subsets/water_input_file_3.tif
m Ocean level         = 0
m Data width  = 1024
m Data height = 3072
m Data cells  = 3145728


Finished.
IO time   = 0.0591588 s
Calc time = 2.35625 s
loading results...
running fill-spill-merge...
m Input DEM           = rema_subsets/dem_small_2.tif
m Output prefix       = fsm_results/rema_tests/test-2
m Surface water level = nan
m Surface water file  = rema_subsets/water_input_file_3.tif
m Ocean level         = 0
m Data width  = 1024
m Data height = 3072
m Data cells  = 3145728


Finished.
IO time   = 0.057004 s
Calc time = 2.50196 s
loading results...


BokehModel(combine_events=True, render_bundle={'docs_json': {'4f837cc7-82c3-4ae5-ab78-c014c19d9c14': {'version…

In [51]:
results.hvplot.scatter(x='melt_mag', y = 'y_center')

BokehModel(combine_events=True, render_bundle={'docs_json': {'c60ee75d-3eb6-4aea-8b11-b56a9f6ec7e1': {'version…

In [ ]:
ds_rechunked_smaller = ds.isel(x=slice(0,25000,),y=slice(0,25000,10)).chunk({'x': 4096, 'y': 4096}).compute()

In [ ]:
ds_rechunked_smaller.to_zarr('/Users/jkingslake/Documents/science/meltwater_routing/big_data/boudouin_west_1_smaller')